Polymer Blend
===========

insert description

### Concepts Used
- blergh

### Tools Used
- blergh

References
-----------
1. Yethiraj, A.; Schweizer, K.S.; Integral equation theory of polymer blends: Numerical Investigation of molecular closure approximations, J. Chem. Phys., 1993, 98 (11) 
 
  

Global Imports and Setup
----------------------------

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh','matplotlib')

%load_ext autoreload
%autoreload 2

In [ ]:
%%opts Overlay [width=600,height=400]
import typyPRISM
domain = typyPRISM.Domain(length=1024,dr=0.01)
curves = []
for i,epsilon in enumerate(np.arange(0.05,0.19,0.025)):
    epsilon*=-1
    U = typyPRISM.potential.HardCoreLennardJones(epsilon=epsilon,sigma=sigma).calculate(domain.r)
    curves.append(hv.Scatter((domain.r,U),label=str(epsilon),extents=(1,-0.1,2,0.3)))
hv.Overlay(curves)

In [ ]:
import typyPRISM
import numpy as np
delta =  0.05#0.076260500979288562
factor = 1.0/(1.0 - delta)

N = 8
sigma = l = 1.0
phi     = 0.5
eta     = 0.3
v1      = 4.0/3.0 * np.pi * (sigma/2.0)**(3.0)
rhoA    = phi * eta/v1*factor
rhoB    = (1-phi) * eta/v1*factor
kT = 1.0
print(rhoA,rhoB)

sys = typyPRISM.System(['A','B'],kT=kT)
sys.domain = typyPRISM.Domain(dr=0.025,length=2048)

sys.density['A'] = rhoA
sys.density['B'] = rhoB

sys.diameter[sys.types] = 1.0

sys.potential[sys.types,sys.types] = typyPRISM.potential.HardSphere(sigma=sigma)
sys.closure[sys.types,sys.types] = typyPRISM.closure.PercusYevick(True)
sys.omega[sys.types,sys.types] = typyPRISM.omega.DiscreteKoyama(lp=4.0/3.0,length=N,sigma=sigma,l=l)
sys.omega['A','B'] = typyPRISM.omega.InterMolecular()

lam_list = []
guess = np.zeros(sys.domain.length*sys.rank*sys.rank)
for i,epsilon in enumerate(np.arange(0.05,0.19,0.01)):
    epsilon *= -1.0
    print('T=',kT/abs(epsilon))
    print('1/T=',abs(epsilon)/kT)

    sys.potential['A','B'] = typyPRISM.potential.HardCoreLennardJones(epsilon=epsilon,sigma=sigma)

    PRISM = sys.createPRISM()
    PRISM.solve(guess,method='krylov')

    lam = typyPRISM.calculate.spinodal_condition(PRISM)
    lam_list.append([abs(epsilon),lam['A','B']])
    
    guess = np.copy(PRISM.x)
    
    print(np.array(lam_list))

np.array(lam_list)

In [ ]:
xy1 = np.array(lam_list)
# xy2 = np.array(lam_list2)
hv.Curve(xy1,extents=(0,0,1.2,30),label='MSA') #* hv.Curve(xy2,extents=(0,0,1.2,30),label='PY')

In [ ]:
xy1 = np.array(lam_list)
# xy2 = np.array(lam_list2)
hv.Curve(xy1,extents=(0,0,1.2,30),label='MSA') #* hv.Curve(xy2,extents=(0,0,1.2,30),label='PY')

In [ ]:
import typyPRISM
import scipy
import numpy as np

def density_correction(sigma,N,domain,omega_real):
    r = domain.r
    
    integrand  =(np.pi/6.0)*(sigma**(3.0)) 
    integrand *=(1 - 3*r/(2*sigma) + 0.5*(r/sigma)**(3.0))
    integrand *= omega_real
    integrand *= 4*np.pi*r*r
    
    mask = r<=sigma
    # integrand[r>sigma] = 0.0 # we only integrate inside the hard core
    delta  = scipy.integrate.simps(integrand[mask],x=r[mask])
    delta *= 1.0/(N*np.pi*sigma*sigma*sigma/6.0)
    
    factor = 1/(1-delta)
    return delta,factor

# domain = typyPRISM.Domain(dr=0.001,length=1001)
domain = typyPRISM.Domain(dr=0.001,length=1024)

N = 2000

l = sigma = 1.0
O = typyPRISM.omega.DiscreteKoyama(lp=4.0/3.0,length=N,sigma=sigma,l=l)
omega_fourier = 2*O.calculate_outer(domain.k) 
omega_real = domain.to_real(omega_fourier)

print(O.lp,np.arccos(O.cos0),np.pi/3)

density_correction(sigma,N,domain,omega_real)

In [ ]:
0.005*1024